In [2]:
# Installing conda
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0   9975      0 --:--:-- --:--:-- --:--:--  9962


add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
INFO:conda_installer:add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
python version: 3.10.12
INFO:conda_installer:python version: 3.10.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
INFO:conda_installer:fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
INFO:conda_installer:done
installing miniconda to /root/miniconda
INFO:conda_installer:installing miniconda to /root/miniconda
done
INFO:conda_installer:done
installing openmm, pdbfixer
INFO:conda_installer:installing openmm, pdbfixer
added conda-forge to channels
INFO:conda_installer:added conda-forge to channels
done
INFO:conda_installer:done
conda packages installation finished!
INFO:conda_installer:conda packages installation finished!


# conda environments:
#
base                     /root/miniconda



In [3]:
# Installing Deepchem
!pip install --pre deepchem
import deepchem
deepchem.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 33.9 MB/s eta 0:00:00


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


'2.8.1.dev'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/BBB dataset/bbb_cns_train.csv')
val=pd.read_csv('/content/drive/MyDrive/BBB dataset/bbb_cns_val.csv')
test=pd.read_csv('/content/drive/MyDrive/BBB dataset/bbb_cns_test.csv')

In [5]:
import deepchem as dc
import numpy as np

In [37]:
featurizer = dc.feat.CircularFingerprint(size=1024)
tasks = ["label"]
smiles_field = "smiles_standarized"
loader = deepchem.data.CSVLoader(tasks=tasks, smiles_field=smiles_field, featurizer=featurizer)
train_dataset = loader.featurize('/content/drive/MyDrive/BBB dataset/bbb_cns_train.csv')
test_dataset = loader.featurize('/content/drive/MyDrive/BBB dataset/bbb_cns_test.csv')
valid_dataset = loader.featurize('/content/drive/MyDrive/BBB dataset/bbb_cns_val.csv')

normalizer = deepchem.trans.NormalizationTransformer(transform_y=True,
                                                         dataset=train_dataset,
                                                         move_mean=True)
train_dataset = normalizer.transform(train_dataset)
test_dataset = normalizer.transform(test_dataset)
valid_dataset = normalizer.transform(valid_dataset)

/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(


In [38]:
print(len(train_dataset))
print(len(test_dataset))
print(len(valid_dataset))

122
15
16


In [39]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
model = dc.models.SklearnModel(model=rf)
# model training
model.fit(train_dataset)
valid_preds = model.predict(valid_dataset)
valid_preds.shape
test_preds = model.predict(test_dataset)
test_preds.shape

(15,)

In [40]:
# initialze the metric
metric = dc.metrics.Metric(dc.metrics.mae_score)
# evaluate the model
train_score = model.evaluate(train_dataset, [metric])
valid_score = model.evaluate(valid_dataset, [metric])
test_score = model.evaluate(test_dataset, [metric])

In [41]:
print(train_score)
print(valid_score)
print(test_score)

{'mae_score': 0.23191208543088607}
{'mae_score': 0.5503908891744722}
{'mae_score': 0.7021004948512111}


In [44]:
# We want to know the pearson R squared score, averaged across tasks
avg_pearson_r2 = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)

# We'll train a multitask regressor (fully connected network)
model = dc.models.MultitaskRegressor(
    len(tasks),
    n_features=1024,
    layer_sizes=[500])

model.fit(train_dataset)


# We now evaluate our fitted model on our training and validation sets
train_scores = model.evaluate(train_dataset, [avg_pearson_r2])
print(train_scores['mean-pearson_r2_score'])
print(train_scores)
assert train_scores['mean-pearson_r2_score'] > 0.7, train_scores

valid_scores = model.evaluate(valid_dataset, [avg_pearson_r2])
print(valid_scores['mean-pearson_r2_score'])
assert valid_scores['mean-pearson_r2_score'] > 0.3, valid_scores

0.8159358963566142
{'mean-pearson_r2_score': 0.8159358963566142}
0.5655605102332006


In [9]:
featurizer = dc.feat.ConvMolFeaturizer()
tasks = ["label"]
smiles_field = "smiles_standarized"
loader = deepchem.data.CSVLoader(tasks=tasks, smiles_field=smiles_field, featurizer=featurizer)
train_dataset = loader.featurize('/content/drive/MyDrive/BBB dataset/bbb_cns_train.csv')
test_dataset = loader.featurize('/content/drive/MyDrive/BBB dataset/bbb_cns_test.csv')
valid_dataset = loader.featurize('/content/drive/MyDrive/BBB dataset/bbb_cns_val.csv')

normalizer = deepchem.trans.NormalizationTransformer(transform_y=True,
                                                         dataset=train_dataset,
                                                         move_mean=True)
avg_pearson_r2 = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)
train_dataset = normalizer.transform(train_dataset)
test_dataset = normalizer.transform(test_dataset)
valid_dataset = normalizer.transform(valid_dataset)

model = dc.models.GraphConvModel(len(tasks), mode='regression', dropout=0.5)

model.fit(train_dataset, nb_epoch=30)
# We now evaluate our fitted model on our training and validation sets
train_scores = model.evaluate(train_dataset, [avg_pearson_r2])

valid_scores = model.evaluate(valid_dataset, [avg_pearson_r2])
print(train_scores)
print(valid_scores)

/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(


{'mean-pearson_r2_score': 0.3032036740321629}
{'mean-pearson_r2_score': 0.3522580868286936}


In [13]:
import deepchem as dc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Define your tasks and CSV paths
tasks = ["label"]
smiles_field = "smiles_standarized"
train_file = '/content/drive/MyDrive/BBB dataset/bbb_cns_train.csv'
valid_file = '/content/drive/MyDrive/BBB dataset/bbb_cns_val.csv'
test_file = '/content/drive/MyDrive/BBB dataset/bbb_cns_test.csv'

# Create the ECFP featurizer and loader
featurizer = dc.feat.CircularFingerprint(size=1024)
loader = dc.data.CSVLoader(tasks=tasks, smiles_field=smiles_field, featurizer=featurizer)

# Featurize datasets
train_dataset = loader.featurize(train_file)
valid_dataset = loader.featurize(valid_file)
test_dataset = loader.featurize(test_file)

# Normalize datasets
normalizer = dc.trans.NormalizationTransformer(transform_y=True, dataset=train_dataset, move_mean=True)
train_dataset = normalizer.transform(train_dataset)
valid_dataset = normalizer.transform(valid_dataset)
test_dataset = normalizer.transform(test_dataset)

# Ensure all dataset features and targets are of type float32 or float64
def convert_dataset_types(dataset):
    dataset.X = dataset.X.astype(np.float32)
    dataset.y = dataset.y.astype(np.float32)
    return dataset

train_dataset = convert_dataset_types(train_dataset)
valid_dataset = convert_dataset_types(valid_dataset)
test_dataset = convert_dataset_types(test_dataset)

# Define the RMS metric
avg_rms = dc.metrics.Metric(dc.metrics.rms_score, np.mean)

# Create the model
n_layers = 3
model = dc.models.MultitaskRegressor(
    len(tasks),
    n_features=1024,  # ECFP size
    layer_sizes=[1000] * n_layers,
    dropouts=[.25] * n_layers,
    weight_init_stddevs=[.02] * n_layers,
    bias_init_consts=[1.] * n_layers,
    learning_rate=.0003,
    weight_decay_penalty=.0001,
    batch_size=100)

# Fit the model
model.fit(train_dataset, nb_epoch=5)

# Evaluate the model
train_scores = model.evaluate(train_dataset, [avg_rms])
valid_scores = model.evaluate(valid_dataset, [avg_rms])

# Print the results
print("Train scores:", train_scores)
print("Validation scores:", valid_scores)


/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/deepchem/data/data_loader.py:173: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  warnings.warn(


AttributeError: can't set attribute 'X'